In [22]:
import numpy as np
import pandas as pd
import pickle as pk

#### As part of preprocessing, the Mels co-effs are extracted from All 21 Labeled Audion file and been pickled in labeled_mel_dict.p.  We have another pickled file which contains the location of the Heart Sound in respective wav files 

In [199]:
fileloc='/Users/spandan.chakraborty/Documents/UML/Capstone/HS/Mycode/Mels/Code/'
filename='labeled_mel_dict.p'
#sound_locations='/Users/spandan.chakraborty/Documents/UML/Capstone/HS/Mycode/Mels/labels/set_a_timing_csv.p'
sound_locations=fileloc+'set_a_timing_csv.p'
### df is a dictionary of All mels co effs (128 mels* duration of the file)
df=pd.read_pickle(fileloc+filename)
### df_sound_loc contains the location of the Heart Sound
df_sound_loc=pd.read_pickle(sound_locations)
#print(type(df))

In [202]:
### Example of a dictionary 
print(df['normal__201108011118'].shape)
### audio info converted into Array (mels_Co_eff)
# each file represents 128 Mel Co effs .. since each file contains variable length of Audio the segment Size varies
df['normal__201108011118']

(128, 684)


array([[  8.04295694e-03,   8.93476285e-02,   4.81661272e-01, ...,
          1.65827122e+00,   1.63090038e+00,   2.36399915e+00],
       [  1.56986214e-02,   1.74392900e-01,   9.40129107e-01, ...,
          3.23669169e+00,   3.18326802e+00,   4.61416465e+00],
       [  2.43042414e-01,   3.93015880e-01,   6.82248374e-01, ...,
          6.73354401e-01,   1.07922115e+00,   3.23077510e+00],
       ..., 
       [  1.87514210e-06,   5.44098180e-07,   1.67955814e-06, ...,
          1.61014473e-07,   2.96333669e-06,   5.41442064e-05],
       [  1.90644479e-07,   1.50682207e-06,   2.07056151e-06, ...,
          5.78843901e-07,   4.55531865e-06,   6.18284918e-05],
       [  4.18645852e-07,   2.09581515e-06,   4.93758600e-06, ...,
          2.27469929e-06,   2.57632735e-06,   3.87966964e-05]])

In [206]:
### Heart Sound Location Labels
df_sound_loc[df_sound_loc.fname.str.contains('normal__201108011118')].head(2)

,fname,cycle,sound,location,TimeInSec,segment
366,set_a/normal__201108011118.wav,1,S1,7591,0.172132,14
367,set_a/normal__201108011118.wav,1,S2,20496,0.464762,40


In [232]:
len(df_sound_loc)

390

In [207]:
df_sound_loc.fname=df_sound_loc.fname.str.replace('set_a/','')
df_sound_loc.fname=df_sound_loc.fname.str.replace('.wav','')

In [208]:
df_sound_loc[df_sound_loc.fname.str.contains('normal__201108011118')].head(2)

,fname,cycle,sound,location,TimeInSec,segment
366,normal__201108011118,1,S1,7591,0.172132,14
367,normal__201108011118,1,S2,20496,0.464762,40


In [205]:
### The Window width for Librosa is 512 samples per segment
### The input label as Given in Kaggle is as per 44100 frames per seconf sample rate
### Converting the labeled location into Segment 
df_sound_loc['segment']=(df_sound_loc['location']/512).astype(int)

### Since HS lasts approx. 0.11- 0.14  secs & 1 sec of wav file contains 87 segments (for 44100 Sample Rate and 512 Hop length (window size  512 samples)).. it turns out the HS will be of 0.14*87  ~= 12 segment long

In [209]:
df_sound_loc['HS_Start']=(df_sound_loc['segment']-6).astype(int)

In [210]:
df_sound_loc['HS_End']=(df_sound_loc['segment']+6).astype(int)

In [212]:
df_sound_loc[df_sound_loc.fname.str.contains('normal__201108011118')].head(2)

,fname,cycle,sound,location,TimeInSec,segment,HS_Start,HS_End
366,normal__201108011118,1,S1,7591,0.172132,14,8,20
367,normal__201108011118,1,S2,20496,0.464762,40,34,46


#### Find the Noise Space in the mels , Want to get same 13 Width Mel time window for noise

the Shift(-1) function returns values from Next Row respective to current row 

In [216]:
#df_sound_loc['noise_window']=df_sound_loc['HS_Start'].shift(-1)-df_sound_loc['HS_End']

#### Extract the noise window between Heart Sounds

In [222]:
df_sound_loc['n_start']=df_sound_loc.groupby('fname')['HS_End'].apply(lambda x: x+1)

In [223]:
# shift(-1) provides the next row reference
df_sound_loc['n_end']=df_sound_loc.groupby('fname')['HS_Start'].shift(-1).apply(lambda x:x-1)

In [229]:
# df_sound_loc[df_sound_loc.fname.str.contains('normal__201108011118')]
# df_sound_loc[df_sound_loc.fname.str.contains('normal__201108011114')]

In [231]:
df_noise=pd.DataFrame()
df_noise=df_sound_loc.copy()
print(len(df_noise))

390


In [235]:
# exclude the lines where n_start-n_end <12 segments
df_noise[(df_noise['n_end']-df_noise['n_start'])<12].index

Int64Index([  0,   2,   4,   6,   8,  10,  12,  14,  16,  18,
            ...
            280, 282, 288, 292, 370, 372, 376, 380, 384, 388],
           dtype='int64', length=111)

In [240]:
### This is an Important Temporal Information I am going to drop, 
### with this the relative Timiing Gaps pattern between HS1 - HS2 -HS1 will be lost 
### But since I am only ineterested in Fifferentiating from HS 1 to HS2 to Noise .. 
### I will drop this info for time being
#df_noise.iloc[df_noise[(df_noise['n_end']-df_noise['n_start'])<12].index].apply(lambda x: x['n_end']-x['n_start'], axis=1)

In [241]:
df_noise.drop(df_noise[(df_noise['n_end']-df_noise['n_start'])<12].index, inplace=True)
len(df_noise)

279

In [247]:
df_noise.dropna(how='any', inplace=True)  ## Any row Has nan value will be dropped
len(df_noise)

258

In [249]:
#df_noise.n_end.unique()

In [256]:
df_noise.n_end=df_noise.n_end.astype(int)

In [261]:
df_noise['nn_end']=df_noise.n_start+12
df_noise.sound='n'
df_noise.head(2)

,fname,cycle,sound,location,TimeInSec,segment,HS_Start,HS_End,n_start,n_end,nn_end
1,normal__201102081321,1,n,20759,0.470726,40,34,46,47,61,59
3,normal__201102081321,2,n,47244,1.071293,92,86,98,99,116,111


In [268]:
df_hs=pd.DataFrame()
df_hs=df_sound_loc.copy()
df_hs.drop(['cycle','location','TimeInSec','segment','n_start','n_end'], axis=1, inplace=True)
df_hs.rename(columns={'HS_Start': 'start','HS_End':'end'}, inplace=True)
df_hs.head(2)

,fname,sound,start,end
0,normal__201102081321,S1,13,25
1,normal__201102081321,S2,34,46


In [269]:
df_noise.drop(['cycle','location','TimeInSec','segment','HS_Start','HS_End','n_end'], axis=1, inplace=True)
df_noise.rename(columns={'n_start':'start','nn_end':'end'}, inplace=True)
df_noise.head(2)

,fname,sound,start,end
1,normal__201102081321,n,47,59
3,normal__201102081321,n,99,111


In [339]:
df_s=pd.concat([df_hs,df_noise], axis=0)
print(len(df_s))
df_s.groupby('sound').size()

648


sound
S1    195
S2    195
n     258
dtype: int64

### Get the Mels value from Dictionary using File names as key through map 
#### The idea is to Slice the 128 mels of 0.14 secs of Slices (Almost 12 segements around the HS labled Locations )

In [340]:
df_s['mels']=df_sound_loc['fname'].map(df)

In [341]:
df_s[df_s.fname.str.contains('normal__201108011118')].head(2)

,fname,sound,start,end,mels
366,normal__201108011118,S1,8,20,"[[0.00804295694391, 0.089347628485, 0.48166127..."
367,normal__201108011118,S2,34,46,"[[0.00804295694391, 0.089347628485, 0.48166127..."


#### Following code used list Slicing fetching the Heart Sound location specific Mels

In [342]:
df_s['mels_seg'] = df_s.apply(lambda x: x['mels'][:, x['start']:x['end']].tolist(),axis=1)

In [343]:
df_s.head(2)

,fname,sound,start,end,mels,mels_seg
0,normal__201102081321,S1,13,25,"[[0.0752865622903, 0.00439239454838, 0.0182232...","[[0.0171469795289, 0.0173154008662, 0.39569554..."
1,normal__201102081321,S2,34,46,"[[0.0752865622903, 0.00439239454838, 0.0182232...","[[0.0471267533454, 0.0061760868171, 0.00564760..."


In [65]:
#df_sound_loc[df_sound_loc.fname.str.contains('normal__201106111136')].head(2)

In [344]:
np.array(df_s.ix[0, 'mels_seg']).shape  ## np.array converts the list to Array 

(128, 12)

In [499]:
df_s.drop('mels', inplace=True)
#df_s.head(2)
S1=np.array([1.0,0.0,0.0])
S2=np.array([0.0,1.0,0.0])
n=np.array([0.0,0.0,1.0])
sound_class={'S1':S1,'S2':S2,'n':n}
df_s['class']=df_s['sound'].map(sound_class)
#df_s['mels_flatten']=df_s['mels_seg'].apply(lambda x: np.asarray(x).flatten())
df_s['mels_flatten']=df_s['mels_seg'].apply(np.ravel)
df_s.head(2)

,fname,sound,start,end,mels,mels_seg,class,mels_flatten
0,normal__201102081321,S1,13,25,"[[0.0752865622903, 0.00439239454838, 0.0182232...","[[0.0171469795289, 0.0173154008662, 0.39569554...","[1.0, 0.0, 0.0]","[0.0171469795289, 0.0173154008662, 0.395695541..."
1,normal__201102081321,S2,34,46,"[[0.0752865622903, 0.00439239454838, 0.0182232...","[[0.0471267533454, 0.0061760868171, 0.00564760...","[0.0, 1.0, 0.0]","[0.0471267533454, 0.0061760868171, 0.005647608..."


#### Check if all the elements in the Dataset is correct 

In [566]:
df_s['mels_flatten'].apply(len).unique()

array([1536,    0])

##### It indicates there are 0 length Data elements in mels_flatten

In [567]:
df_s[df_s['mels_flatten'].apply(len)==0].index

Int64Index([136, 294], dtype='int64')

In [568]:
df_s.iloc[[136,294]]

,fname,sound,start,end,mels,mels_seg,class,mels_flatten
136,normal__201105011626,S1,-1,11,"[[0.147678276527, 0.040960011283, 0.0059928980...","[[], [], [], [], [], [], [], [], [], [], [], [...","[1.0, 0.0, 0.0]",[]
294,normal__201106221450,S1,-1,11,"[[0.000655332564528, 0.0046009260886, 0.067146...","[[], [], [], [], [], [], [], [], [], [], [], [...","[1.0, 0.0, 0.0]",[]


In [569]:
## drop the rogue elements 
df_s.drop(df_s.index[[136,294]], inplace= True)

In [570]:
df_s['mels_flatten'].apply(len).unique()

array([1536])

In [571]:
df_s.groupby('sound').size()

sound
S1    193
S2    195
n     257
dtype: int64

In [635]:
df_num={'S1':1,'S2':2,'n':3}
df_s['sound_num']=df_s['sound'].map(df_num)
df_s.groupby('sound_num').size()

sound_num
1    193
2    195
3    257
dtype: int64

In [577]:
### Establish Baseline for a MultiClass Classification : Zero Rule Classification
## Go by Simple Targer Data Distribution:
#print(len(df_s))
print 'Error Rate if we always predict Majority (which is noise):',(193+195)/(193.+195+257)


Error Rate if we always predict Majority (which is noise): 0.601550387597


In [730]:
from sklearn.model_selection import train_test_split
X_All=df_s.ix[:,['mels_flatten']]
y_All=df_s.ix[:,['class']] 
# instead of the one hot encoded lets take the label for sound to match the input of tf mnist example
#y=df_s.ix[:,['sound_num']] 
print(X_All.size, y_All.size)

645 645


In [731]:
X_All.head(2)

,mels_flatten
0,"[0.0171469795289, 0.0173154008662, 0.395695541..."
1,"[0.0471267533454, 0.0061760868171, 0.005647608..."


In [732]:
y_All['class'].apply(len).unique()

array([3])

In [733]:
#X_inp=np.vstack(X['mels_flatten'].values).reshape(X['mels_flatten'].shape[0], 128, 12,1).astype(np.float32)
X_inp_flat=np.vstack(X_All['mels_flatten'].values) ## This one is used to pass a flat array to tf
X_inp_flat=X_inp_flat.astype(np.float32, copy=False)
y_inp=np.vstack(y_All['class'].values)
#print(X_inp.shape)
print(X_inp_flat.shape)
print(y_inp.shape)

(645, 1536)
(645, 3)


In [734]:
print (X_inp_flat[1].shape)
print (y_inp[1])

(1536,)
[ 0.  1.  0.]


In [832]:
X_train_s, X_test, y_train_s, y_test = train_test_split(X_inp_flat, y_inp, test_size=0.15, random_state=42)

In [833]:
print(y_train_s.shape, y_test.shape)

(548, 3) (97, 3)


In [737]:
# X_train, X_val_df, y_train, y_val_df = train_test_split(X_train_s, y_train_s, test_size=0.15, random_state=42)
# print(y_train.shape, y_val_df.shape)

(438, 3) (78, 3)


### Following TF Mnist set up:
https://www.tensorflow.org/get_started/mnist/pros#about_this_tutorial

#### load MNIST Data just for comparision of my array input

In [690]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


#### Start TensorFlow InteractiveSession

In [963]:
import tensorflow as tf
sess = tf.InteractiveSession()

#### Placeholders: We start building the computation graph by creating nodes for the input images and target output classes.

In [964]:
x = tf.placeholder(tf.float32, shape=[None, 1536], name='x_variable')
y_ = tf.placeholder(tf.float32, shape=[None, 3],name='y_variable')

Here x and y_ aren't specific values. Rather, they are each a placeholder -- a value that we'll input when we ask TensorFlow to run a computation.

The input images x will consist of a 2d tensor of floating point numbers. Here we assign it a shape of [None, 1536], where 1536 is the dimensionality of a single flattened 128 mels by 12 segments Audio attributes, and None indicates that the first dimension, corresponding to the batch size, can be of any size. The target output classes y_ will also consist of a 2d tensor, where each row is a one-hot 3-dimensional vector indicating which digit class (1 to 3 corresponding to S1, S2 and Noise) the corresponding Heart Sound the audio section belongs to.

The shape argument to placeholder is optional, but it allows TensorFlow to automatically catch bugs stemming from inconsistent tensor shapes.

#### Variables

We now define the weights W and biases b for our model. We could imagine treating these like additional inputs, but TensorFlow has an even better way to handle them: Variable. A Variable is a value that lives in TensorFlow's computation graph. It can be used and even modified by the computation. In machine learning applications, one generally has the model parameters be Variables.

In [965]:
# All variables are initialized with zeros 
W = tf.Variable(tf.zeros([1536,3]),name='weights')
b = tf.Variable(tf.zeros([3]),name='biases')

Before Variables can be used within a session, they must be initialized using that session. This step takes the initial values (in this case tensors full of zeros) that have already been specified, and assigns them to each Variable. This can be done for all Variables at once:

In [966]:
sess.run(tf.global_variables_initializer())

Predicted Class and Loss Function

In [967]:
y = tf.matmul(x,W) + b
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y), name='cross_entropy')

### Train the Model
Now that we have defined our model and training loss function, it is straightforward to train using TensorFlow. Because TensorFlow knows the entire computation graph, it can use automatic differentiation to find the gradients of the loss with respect to each of the variables. TensorFlow has a variety of built-in optimization algorithms. For this example, we will use steepest gradient descent, with a step length of 0.5, to descend the cross entropy.

In [968]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy, name='train_step')

#### Run the model 

Since our train Size is very small we can train the model in one run 

In [969]:
#train_step.run(feed_dict={x: X_train, y_: y_train})
#train_step.run(feed_dict={x: X_train_s, y_: y_train_s})
sess.run(train_step, feed_dict={x: X_train_s, y_: y_train_s})

### Evaluate the Model
tf.argmax is an extremely useful function which gives you the index of the highest entry in a tensor along some axis. For example, tf.argmax(y,1) is the label our model thinks is most likely for each input, while tf.argmax(y_,1) is the true label. We can use tf.equal to check if our prediction matches the truth.

In [970]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1),name='prediction')


In [971]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='Calc_Accuracy')
print(accuracy.eval(feed_dict={x: X_train, y_: y_train}))

0.376712


#### Accuracy on Test Set:

In [972]:
print(accuracy.eval(feed_dict={x: X_test, y_: y_test}))

0.360825


In [973]:
assert sess is tf.get_default_session()
sess.close()

In [974]:
tf.get_default_session()

# Build a Multilayer Convolutional Network


#### Weight Initialization

To create this model, we're going to need to create a lot of weights and biases. One should generally initialize weights with a small amount of noise for symmetry breaking, and to prevent 0 gradients. Since we're using ReLU neurons, it is also good practice to initialize them with a slightly positive initial bias to avoid "dead neurons". Instead of doing this repeatedly while we build the model, let's create two handy functions to do it for us.

In [1007]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.01)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#### Convolution and Pooling
TensorFlow also gives us a lot of flexibility in convolution and pooling operations. How do we handle the boundaries? What is our stride size? In this example, we're always going to choose the vanilla version. Our convolutions uses a stride of one and are zero padded so that the output is the same size as the input. Our pooling is plain old max pooling over 2x2 blocks. To keep our code cleaner, let's also abstract those operations into functions.

In [1008]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

#### First Convolutional Layer

We can now implement our first layer. It will consist of convolution, followed by max pooling. The convolution will compute 32 features for each 5x5 patch. Its weight tensor will have a shape of [5, 5, 1, 32]. The first two dimensions are the patch size, the next is the number of input channels, and the last is the number of output channels. We will also have a bias vector with a component for each output channel.

In [1009]:
#W_conv1 = weight_variable([5, 5, 1, 32])
W_conv1 = weight_variable([2, 6, 1, 32])
b_conv1 = bias_variable([32])

To apply the layer, we first reshape x to a 4d tensor, with the second and third dimensions corresponding to Audio Mels Co efficients and Duration converted into Segements (87 segments corresponds to 1 sec of audio in .wav format), and the final dimension for image processing corresponds to the number of color channels, but in my case I just kept it as 1.

In [1010]:
x_audio = tf.reshape(x, [-1, 128, 12, 1])

We then convolve x_image with the weight tensor, add the bias, apply the ReLU function, and finally max pool. The max_pool_2x2 method will reduce the image size to 64x6.

In [1011]:
h_conv1 = tf.nn.relu(conv2d(x_audio, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

#### Second Convolutional Layer

In order to build a deep network, we stack several layers of this type. The second layer will have 64 features for each 5x5 patch.

In [1012]:
W_conv2 = weight_variable([5, 5, 32, 64])
#W_conv2 = weight_variable([32, 8, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

##### Densely Connected Layer

Now that the image size has been reduced to 7x7, we add a fully-connected layer with 1024 neurons to allow processing on the entire image. We reshape the tensor from the pooling layer into a batch of vectors, multiply by a weight matrix, add a bias, and apply a ReLU.

In [1013]:
W_fc1 = weight_variable([32 * 3 * 64, 1024])
#W_fc1 = weight_variable([32 * 3 * 64, 2048])
b_fc1 = bias_variable([1024])
#b_fc1 = bias_variable([2048])

h_pool2_flat = tf.reshape(h_pool2, [-1, 32*3*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#### Dropout

To reduce overfitting, we will apply dropout before the readout layer. We create a placeholder for the probability that a neuron's output is kept during dropout. This allows us to turn dropout on during training, and turn it off during testing. TensorFlow's tf.nn.dropout op automatically handles scaling neuron outputs in addition to masking them, so dropout just works without any additional scaling.1

In [1014]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#### Readout Layer

Finally, we add a layer, just like for the one layer softmax regression above

In [1015]:
W_fc2 = weight_variable([1024, 3])
#W_fc2 = weight_variable([2048, 3])
b_fc2 = bias_variable([3])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [1016]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv), name ='Conv_Cross_Entropy')
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1),name ='Conv_Accuracy')
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [1033]:
from tqdm import tqdm

In [1036]:
with tf.Session() as sess1:
    assert sess1 is tf.get_default_session()
    sess1.run(tf.global_variables_initializer())
    
    for i in tqdm(range(2000)):
        sess1.run(train_step,feed_dict={x: X_train_s, y_: y_train_s, keep_prob: 1.})
    
    train_accuracy = accuracy.eval(feed_dict={x: X_train_s, y_: y_train_s, keep_prob: 1.})
    print('training accuracy', train_accuracy)
    
#     validiation_accuracy = accuracy.eval(feed_dict={x: X_val_df, y_: y_val_df,keep_prob: 1.})
#     print('validation accuracy', validiation_accuracy)
    
    test_accuracy = accuracy.eval(feed_dict={x: X_test, y_: y_test,keep_prob: 1.})
    print('Test accuracy', test_accuracy)

100%|██████████| 2000/2000 [54:04<00:00,  1.68s/it] 


training accuracy 0.985401
Test accuracy 0.762887


#### With Higher no# of Epochs and with Keep Proba as 0.5 while training in a different session
Keep Proba = 0.5 to ensure we are not overfitting to the Training Data, while Training in Train Step

In [1037]:
with tf.Session() as sess2:
    assert sess2 is tf.get_default_session()
    sess2.run(tf.global_variables_initializer())
    for i in tqdm(range(2500)):
        sess2.run(train_step,feed_dict={x: X_train_s, y_: y_train_s, keep_prob: .5})
    train_accuracy = accuracy.eval(feed_dict={x: X_train_s, y_: y_train_s, keep_prob: 1.})
    print('training accuracy', train_accuracy)
    
#     validiation_accuracy = accuracy.eval(feed_dict={x: X_val_df, y_: y_val_df,keep_prob: 1.})
#     print('validation accuracy', validiation_accuracy)
    
    test_accuracy = accuracy.eval(feed_dict={x: X_test, y_: y_test,keep_prob: 1.})
    print('Test accuracy', test_accuracy)

100%|██████████| 2500/2500 [1:10:19<00:00,  1.65s/it]


training accuracy 0.989051
Test accuracy 0.773196


#### Trying out Different Filter sizes in a Third Session 

In [1039]:
W_conv12 = weight_variable([5, 5, 1, 32])
#W_conv1 = weight_variable([2, 6, 1, 32])
b_conv12 = bias_variable([32])

In [1040]:
h_conv12 = tf.nn.relu(conv2d(x_audio, W_conv12) + b_conv12)
h_pool12 = max_pool_2x2(h_conv12)

In [1041]:
W_conv22 = weight_variable([5, 5, 32, 64])
#W_conv2 = weight_variable([32, 8, 32, 64])
b_conv22 = bias_variable([64])

In [1042]:
h_conv22 = tf.nn.relu(conv2d(h_pool12, W_conv22) + b_conv22)
h_pool22 = max_pool_2x2(h_conv22)

In [1043]:
W_fc12 = weight_variable([32 * 3 * 64, 1024])
#W_fc12 = weight_variable([32 * 3 * 64, 2048])
b_fc12 = bias_variable([1024])
#b_fc12 = bias_variable([2048])

In [1044]:
h_pool22_flat = tf.reshape(h_pool22, [-1, 32*3*64])
h_fc12 = tf.nn.relu(tf.matmul(h_pool22_flat, W_fc12) + b_fc12)

In [1045]:
keep_prob = tf.placeholder(tf.float32)
h_fc12_drop = tf.nn.dropout(h_fc12, keep_prob)

In [1046]:
W_fc22 = weight_variable([1024, 3])
#W_fc22 = weight_variable([2048, 3])
b_fc22 = bias_variable([3])

In [1048]:
y_conv2 = tf.matmul(h_fc12_drop, W_fc22) + b_fc22

cross_entropy2 = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv2), name ='Conv_Cross_Entropy2')
train_step2 = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy2)
correct_prediction2 = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1),name ='Conv_Accuracy2')
accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2, tf.float32))

In [1049]:
with tf.Session() as sess3:
    assert sess3 is tf.get_default_session()
    sess3.run(tf.global_variables_initializer())
    
    for i in tqdm(range(2000)):
        sess3.run(train_step2,feed_dict={x: X_train_s, y_: y_train_s, keep_prob: 1.})
    
    train_accuracy2 = accuracy2.eval(feed_dict={x: X_train_s, y_: y_train_s, keep_prob: 1.})
    print('training accuracy', train_accuracy2)
    
#     validiation_accuracy = accuracy.eval(feed_dict={x: X_val_df, y_: y_val_df,keep_prob: 1.})
#     print('validation accuracy', validiation_accuracy)
    
    test_accuracy = accuracy2.eval(feed_dict={x: X_test, y_: y_test,keep_prob: 1.})
    print('Test accuracy', test_accuracy)

100%|██████████| 2000/2000 [58:57<00:00,  1.74s/it] 


training accuracy 0.985401
Test accuracy 0.783505


In [1053]:
with tf.Session() as sess4:
    assert sess4 is tf.get_default_session()
    sess4.run(tf.global_variables_initializer())
    
    for i in tqdm(range(3000)):
        sess4.run(train_step2,feed_dict={x: X_train_s, y_: y_train_s, keep_prob: .5})
    
    train_accuracy2 = accuracy2.eval(feed_dict={x: X_train_s, y_: y_train_s, keep_prob: 1.})
    print('training accuracy', train_accuracy2)
    
#     validiation_accuracy = accuracy.eval(feed_dict={x: X_val_df, y_: y_val_df,keep_prob: 1.})
#     print('validation accuracy', validiation_accuracy)
    
    test_accuracy = accuracy2.eval(feed_dict={x: X_test, y_: y_test,keep_prob: 1.})
    print('Test accuracy', test_accuracy)

100%|██████████| 3000/3000 [1:57:30<00:00,  1.84s/it]  


training accuracy 0.989051
Test accuracy 0.783505


## DO NOT REFER  - CODE NOT IN USE: 
### Define the model

In [633]:
# from __future__ import absolute_import
# from __future__ import division
# from __future__ import print_function
# import tensorflow as tf
# from tensorflow.contrib import learn
# from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

# tf.logging.set_verbosity(tf.logging.INFO)

In [681]:
# def cnn_model_fn(features,labels,mode):
#     # model function for CNN
#     # Input Layer
#     input_layer=tf.reshape(features,[-1,128,12,1]) # The first -1 means it is Dynamic Value to be passed based on 
#     # no# of records
    
#      # Convolutional Layer #1
#     #1st Layer of Activation based filtering 
#     conv1 = tf.layers.conv2d(inputs=input_layer,filters=32,kernel_size=[5, 5],padding="same",activation=tf.nn.relu)
#     # with same Padding the Output is still 128*12 space for each of 32 features extracted = 128*12*32
    
#     # Pooling Layer #1  - downsamples the Conv layers output by taking Max of the value with in the filter window
#     pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
#     # With Pooling Layer#1  2x2 down sampling with stride =2 we have now 64*6 space for each of the 32 features
#     # 64*6*32 
    
#     # Convolutional Layer #2 and Pooling Layer #2
#     conv2 = tf.layers.conv2d(inputs=pool1,filters=64,kernel_size=[5, 5],padding="same",activation=tf.nn.relu)
#     # Output is 64*6 space for each 64 features = 64*6*64
#     pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
#     # Output is 32*3 space for each 64 features = 32*3*64
    
    
#     # Dense Layer
#     #pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
#     pool2_flat = tf.reshape(pool2, [-1, 32 * 3 * 64])
#     dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
#     dropout = tf.layers.dropout(
#     inputs=dense, rate=0.4, training=mode == learn.ModeKeys.TRAIN)
    
#     # Logits Layer
#     logits = tf.layers.dense(inputs=dropout, units=3)

#     loss = None
#     train_op = None
    
#     # Calculate Loss (for both TRAIN and EVAL modes)
#     if mode != learn.ModeKeys.INFER:
#         onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=3)
#         loss = tf.losses.softmax_cross_entropy(
#         onehot_labels=onehot_labels, logits=logits)
        
#     # Configure the Training Op (for TRAIN mode)
#     if mode == learn.ModeKeys.TRAIN:
#         train_op = tf.contrib.layers.optimize_loss(
#             loss=loss,
#             global_step=tf.contrib.framework.get_global_step(),
#             learning_rate=0.001,
#             optimizer="SGD")
    
#     # Generate Predictions
#     predictions = {"classes": tf.argmax(input=logits, axis=1),"probabilities": tf.nn.softmax(logits, name="softmax_tensor")}

#     # Return a ModelFnOps object
#     return model_fn_lib.ModelFnOps(mode=mode, predictions=predictions, loss=loss, train_op=train_op)


